In [60]:
import cv2
import numpy as np

# Create a window to display the video feed
cv2.namedWindow('Webcam_1')
cv2.setMouseCallback('Webcam_1', lambda event, x, y, flags, param: onMouse1(event, x, y, flags, param))

# Initialize the polygon vertices list and current polygon list
polygons_1 = []
current_polygon_1 = []

def onMouse1(event, x, y, flags, param):
    global current_polygon_1

    if event == cv2.EVENT_LBUTTONDOWN:
        current_polygon_1.append((x,y))

    elif event == cv2.EVENT_RBUTTONDOWN:
        if len(current_polygon_1) > 2:
            polygons_1.append(current_polygon_1)
            current_polygon_1 = []

# Initialize the webcam capture object
cap_1 = cv2.VideoCapture(0)

# Start capturing webcam input
while True:
    ret_1, frame_1 = cap_1.read()

    # Draw all polygons on top of the video feed
    for polygon in polygons_1:
        pts_1 = np.array(polygon, np.int32)
        pts_1 = pts_1.reshape((-1,1,2))
        cv2.polylines(frame_1, [pts_1], True, (0,255,0))

    # Draw the current polygon on top of the video feed
    if len(current_polygon_1) > 0:
        pts_1 = np.array(current_polygon_1, np.int32)
        pts_1 = pts_1.reshape((-1,1,2))
        cv2.polylines(frame_1, [pts_1], True, (0,0,255))

    # Display the video feed for each camera
    cv2.imshow('Webcam_1', frame_1)

    # Check for key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam
cap_1.release()

# Calculate the average HSV values of the pixels within each polygon
avg_hsv_array_1 = np.zeros((9, 3))  # Un array de 24 filas y 3 columnas para los valores HSV

hsv_1 = cv2.cvtColor(frame_1, cv2.COLOR_BGR2HSV)
for i, polygon in enumerate(polygons_1):
    pts = np.array(polygon, np.int32)
    pts = pts.reshape((-1,1,2))
    mask = np.zeros_like(frame_1[:,:,0])
    cv2.fillPoly(mask, [pts], 255)

    # Create a mask where the lines are black
    line_mask = np.zeros_like(frame_1[:,:,0])
    cv2.polylines(line_mask,[pts],True,(255,255,255))

    # Combine the polygon mask and line mask to create a new mask
    mask = cv2.bitwise_xor(mask, line_mask)

    # Calculate the average HSV values using the new mask
    avg_hsv = cv2.mean(hsv_1, mask=mask)
    
    # Guardar los valores de avg_hsv en el array
    avg_hsv_array_1[i] = avg_hsv[:3]  # Tomamos solo los primeros tres elementos (H, S, V)

    print(f"Polygon webcam_1 {i}: {avg_hsv}")
    #avg_hsv_array_1 donde se guardan las variables hsv

cv2.destroyAllWindows()

Polygon webcam_1 0: (155.0204081632653, 15.537414965986393, 253.5986394557823, 0.0)
Polygon webcam_1 1: (75.17721518987342, 255.0, 255.0, 0.0)
Polygon webcam_1 2: (0.28125, 255.0, 180.8125, 0.0)
Polygon webcam_1 3: (107.11570247933885, 253.37190082644628, 227.87603305785126, 0.0)
Polygon webcam_1 4: (11.472906403940886, 164.67980295566502, 255.0, 0.0)
Polygon webcam_1 5: (35.03448275862069, 236.20689655172413, 11.793103448275861, 0.0)


In [61]:
import numpy as np
import cv2

array_final_color = avg_hsv_array_1

color_array = np.full((48,), "Desconocido", dtype=object)

def get_limits(hsvC):
    hue = hsvC[0][0][0]  # Get the hue value

    # Handle red hue wrap-around
    if hue >= 165:  # Upper limit for divided red hue
        lowerLimit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upperLimit = np.array([180, 255, 255], dtype=np.uint8)
    elif hue <= 15:  # Lower limit for divided red hue
        lowerLimit = np.array([0, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 10, 255, 255], dtype=np.uint8)
    else:
        lowerLimit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 10, 255, 255], dtype=np.uint8)

    return lowerLimit, upperLimit


def determinar_color(hsvC):
    # Convertir los valores de hsvC de [0, 360] a [0, 180] para la matiz
    hsvC[:, :, 0] = hsvC[:, :, 0] / 2
    #hsvC = hsvC.astype(np.uint8)

    lowerLimit, upperLimit = get_limits(hsvC)
    color = "Desconocido"
    print(hsvC)
    # Definir los rangos de color en HSV
    colores = {
        'azul': (np.array([50, 100, 100]), np.array([120, 255, 255])),
        'naranja': (np.array([4, 100, 100]), np.array([10, 255, 255])),
        'negro': (np.array([0, 0, 0]), np.array([255, 255, 25])),
        'blanco': (np.array([0, 0, 150]), np.array([180, 50, 255])),
        'verde': (np.array([30, 100, 100]), np.array([50, 255, 255])),
        'rojo': (lowerLimit, upperLimit)
    }

#blanco [[[ 50.645454 135.18182  254.      ]]]
# color: 1 azul
# [[[ 51.216667 225.2      203.26666 ]]]
# color: 2 azul
# [[[ 50.563637 137.21819  254.18182 ]]]
# color: 3 azul





    for nombre_color, (limite_inferior, limite_superior) in colores.items():
        if cv2.inRange(hsvC, limite_inferior, limite_superior).any():
            # print(hsvC)
            color = nombre_color
            break

    return color

# Ejemplo de uso
if __name__ == "__main__":
    # Simular un valor de hsvC
    i = 0
    for valor in array_final_color:
        #print(hsvC)
        hsvC = np.array([[valor]], dtype=np.float32)  # Convertir a matriz de numpy
        # Determinar el color predominante
        color_predominante = determinar_color(hsvC)
        color_array[i] = color_predominante
        print("color:",i + 1, color_array[i])
        i = i + 1 
    color_array = np.insert(color_array, 4, 'blanco')
    color_array = np.insert(color_array, 22, 'verde')
    color_array = np.insert(color_array, 49, 'rojo')
    color_array = np.insert(color_array, 31, 'azul')
    color_array = np.insert(color_array, 40, 'naranja')
    color_array = np.insert(color_array, 13, 'negro')



    # color_array[4] = 'blanco'
# color_array[22] = 'verde'
# color_array[44] = 'rojo'
# color_array[31] = 'azul'
# color_array[40] = 'naranja'
# color_array[13] = 'amarrillo'

[[[ 77.5102    15.537415 253.59863 ]]]
color: 1 blanco
[[[ 37.588608 255.       255.      ]]]
color: 2 verde
[[[1.406250e-01 2.550000e+02 1.808125e+02]]]
color: 3 rojo
[[[ 53.55785 253.3719  227.87604]]]
color: 4 azul
[[[  5.736453 164.67981  255.      ]]]
color: 5 naranja
[[[ 17.51724  236.2069    11.793103]]]
color: 6 negro
[[[0. 0. 0.]]]
color: 7 negro
[[[0. 0. 0.]]]
color: 8 negro
[[[0. 0. 0.]]]
color: 9 negro
